# Pretraining
Pretrains the network for the ReverberatorEstimator notebook with given parameters.
These parameters are estimated by listening to the target audio and doing a quick and rough tuning.

In [ ]:
import matplotlib.pyplot as plt
import IPython
import tensorflow as tf
print(tf.__version__)
import tensorflow.keras as tfk
from pedalboard import load_plugin
from ReverberatorEstimator import layers, models, config, utils
import warnings
warnings.filterwarnings('ignore')
import time

In [ ]:
k = config.k
sample_rate = k['sample_rate']
sample_length = k['sample_length']
num_epochs = 5000
num_processors = k['n_processors']
steps_per_epoch = 1
batch_size = 1
epsilon = k['epsilon']
learning_rate = k['learning_rate']
dry_audio_path = k['dry_audio_path']
wet_audio_path = k['wet_audio_path']
vst_path = k['vst_path']
time_loss_weight = k['time_loss_weight']
spectral_loss_weight = k['spectral_loss_weight']
envelope_loss_weight = k['envelope_loss_weight']
echo_density_loss_weight = k['echo_density_loss_weight']
num_params = k['n_parameters']
parameter_map = k['parameter_map']
non_trainable_parameters = k['non_trainable_parameters']
pretrained_weights = k['pretrained_weights']

In [ ]:
vst = load_plugin(vst_path)

In [ ]:
x_train, y_train = utils.get_dataset(dry_audio_path, wet_audio_path, batch_size, resample=True, old_sample_rate=48000, new_sample_rate=sample_rate)

In [ ]:
target_parameters_list = [0.5, 0.9, 0.9]
attenuation_gain_params = [0.9, 0.9, 0.9, 0.85, 0.75, 0.65, 0.50, 0.25, 0.10, 0.10]
tonal_gain_params = [0.9, 0.9, 0.9, 0.9, 0.9, 0.75, 0.65, 0.50, 0.50, 0.50]
bc_gains = [0.70, 0.70]

target_parameters_list = target_parameters_list + attenuation_gain_params + tonal_gain_params
for i in range(16):
    target_parameters_list = target_parameters_list + bc_gains

target_parameters = tf.convert_to_tensor(target_parameters_list, dtype=tf.float32)
target_parameters = tf.reshape(target_parameters, [1, num_params])

In [ ]:
model, parameter_model, _ = models.get_models(sample_length, sample_rate, num_params, 1, 
                                                vst_path, epsilon, parameter_map, non_trainable_parameters)


optimizer = tfk.optimizers.Adam(learning_rate=0.001) 
parameter_model.compile(optimizer=optimizer, loss='mae', run_eagerly=True)

In [ ]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                              factor=0.2,
                              patience=100,
                              cooldown=1,
                              verbose=1,
                              mode='auto',
                              min_lr=0.000001)

In [ ]:
start_time = time.time()
history = parameter_model.fit(x_train, target_parameters, verbose=1, epochs=num_epochs, batch_size=batch_size,
         callbacks=[lr_callback])
print("Training took %d seconds" % (time.time() - start_time))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7,10))
ax[0].plot(history.history['loss'])
ax[0].set_title('loss')
ax[0].set_ylabel('loss')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'test'], loc='upper left')

In [ ]:
parameters = parameter_model(tf.reshape(x_train[0], (1, sample_length)))

In [ ]:
params = parameters.numpy()[0]
print(params)

In [ ]:
param_diff = params - target_parameters_list
plt.stem(param_diff)
plt.ylim([-1, 1])

In [ ]:
parameter_model.save_weights(pretrained_weights)

In [ ]:
for i in range(len(params)):
    parameter_idx = parameter_map[i]
    parameter_value = params[i]
    layers.set_parameter(vst, parameter_idx, parameter_value)
output = vst.process(x_train[0], sample_rate)
IPython.display.display(IPython.display.Audio(output, rate=sample_rate))
plt.plot(output)